# combine BIO tags to spans

# [Dataset format](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune_gemini/tune-gemini-learn#dataset_format)


```jsonl
{
  "systemInstruction": {
    "parts": [
      {
        "text": "(see prompt cell)"
      }
    ]
  },
  "contents": [
    {
      "parts": [
        {
          "text": "In this study, we investigated the effects of aspirin on human platelet function, as well as its interactions with the TP53 gene in Homo sapiens cells in cell lines such as NCI-H1975."
        }
      ]
    },
    {
      "parts": [
        {
          "text": "[\n  {\n    \"text\": \"aspirin\",\n    \"label\": \"drug_chemical\",\n    \"begin\": 46,\n    \"end\": 53\n  },\n  {\n    \"text\": \"human\",\n    \"label\": \"species\",\n    \"begin\": 57,\n    \"end\": 62\n  },\n  {\n    \"text\": \"platelet\",\n    \"label\": \"cell_type\",\n    \"begin\": 63,\n    \"end\": 71\n  },\n  {\n    \"text\": \"TP53\",\n    \"label\": \"gene_protein\",\n    \"begin\": 119,\n    \"end\": 123\n  },\n  {\n    \"text\": \"Homo sapiens\",\n    \"label\": \"species\",\n    \"begin\": 132,\n    \"end\": 144\n  },\n  {\n    \"text\": \"NCI-H1975\",\n    \"type\": \"cell_line\",\n    \"begin\": 173,\n    \"end\": 183\n  }\n]"
        }
      ]
    },
  ]
}

# Prompts
You are a highly specialized biomedical Named Entity Recognition (NER) model. Your task is to accurately identify and extract mentions of biomedical entities from text.

Your goal is to identify and classify entities into one of the following eight types:

- cell_line
- cell_type
- disease
- dna
- rna
- drug_chemical
- gene_protein
- species

For each identified entity, you must output a JSON object containing the following keys:

- "entity": The extracted entity text.
- "type": The entity type from the list above.
- "begin": The starting character index of the entity in the input text (0-indexed).
- "end": The ending character index of the entity in the input text (exclusive, i.e., the index of the character immediately following the entity).

Your output should be a JSON list of these objects, where each object represents a single entity extracted from the input text. If no entities are found, return an empty JSON list.

If no entities are found in the text, return an empty list as follows:
```json
{"entities": []}

Your output must be in a valid JSON format with the following structure:
```json
{
  "entities": [
    {
      "entity": "extracted_entity_text",
      "type": "entity_type",
      "begin": start_index,
      "end": end_index
    },
    ...
  ]
}

Do not include any additional commentary or text in your output. Ensure that the JSON is well-formed and follows exactly the format specified above.

Example:

Input Text:
"In this study, we investigated the effects of aspirin on human platelet function, as well as its interactions with the TP53 gene in Homo sapiens cells in cell lines such as NCI-H1975."

Output JSON:
```json
[
  {
    "text": "aspirin",
    "label": "drug_chemical",
    "begin": 46,
    "end": 53
  },
  {
    "text": "human",
    "label": "species",
    "begin": 57,
    "end": 62
  },
  {
    "text": "platelet",
    "label": "cell_type",
    "begin": 63,
    "end": 71
  },
  {
    "text": "TP53",
    "label": "gene_protein",
    "begin": 119,
    "end": 123
  },
  {
    "text": "Homo sapiens",
    "label": "species",
    "begin": 132,
    "end": 144
  },
  {
    "text": "NCI-H1975",
    "type": "cell_line",
    "begin": 173,
    "end": 183
  }
]

# References
1. [BERN2: an advanced neural biomedical named entity recognition and normalization tool](https://arxiv.org/abs/2201.02080)
1. [PromptNER: Prompting For Named Entity Recognition](https://arxiv.org/abs/2305.15444)
1. [GPT-NER: Named Entity Recognition via Large Language Models](https://arxiv.org/abs/2304.10428)

